### Diplom thesis
- vyzkouset/najit spravnou **architekturu neuronove site** (LSTM 2-3 vnitrni vrstvy, droup out vyzkouset, zmensovani velikosti vrstev, hyperopt, pouzit Keras?)
- **optimalizace parametru** neuronove site
- provest **uceni neuronove site** (to mozna nechat na konec, provest 20x-30x beh nebo cross validaci -> zprumerovat? at je to pouzitelne ono to vychazi dost ruzne)

In [1]:
# keras module for building LSTM 
import pandas as pd
import numpy as np
import string, os 
import math
from matplotlib import pyplot as plt
import re
import ast
import itertools
import pickle
from functools import partial
from collections import Counter
from sklearn.metrics import r2_score

from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast
from SmilesPE.pretokenizer import atomwise_tokenizer
import codecs
from SmilesPE.tokenizer import *
from SmilesPE.spe2vec import *
from SmilesPE.pretokenizer import kmer_tokenizer

import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, SpatialDropout1D

from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor 

import keras.utils as ku 
from keras import models 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from hyperopt.pyll.base import scope 
from hyperopt import pyll, hp, Trials, fmin, tpe, STATUS_OK


import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load data

In [2]:
df = pd.read_csv('../data/VEGF2_ChEMBL28-10980_pic50_noDuplicate_canSmiles_sln_deepSmiles_selfies_inchi.csv',sep=',')

linearNotation_names = ['canonical_smiles','sln','deep_smiles','selfies','inchi']
for notName in linearNotation_names:
    df[notName+'_length'] = df[notName].str.len()
print(df.shape)
# canonical_smiles_length
dfClean = df.loc[(df['canonical_smiles_length'] >30) & (df['canonical_smiles_length']<80)]
print(dfClean.shape)

(7159, 11)
(7044, 11)


### Pomocne funkce

In [3]:
# tady e treba doplnit do vybrane delky, pridavame rovnou cely list (jako token)
def cutData(x,cut_length):
    # jednotlive tokeny jsou zakodovany do delky 100
    lengthTokenSpe=100
    # vytvorim vnulovy vektor
    zeroVec= np.array([[0] * lengthTokenSpe])
    numberTokens = len(x)

    if(cut_length <= numberTokens):
        res=np.array([np.array(i) for i in x[:cut_length]])
    else:
        res = np.array([np.array(i) for i in x])

        for i in range(numberTokens,cut_length):
            res = np.append(res,zeroVec, axis = 0)

    if(len(res)!=cut_length):
        print('delka\n',len(res))
        print(res)
    return res

In [4]:
def tokenizeChemNotation(chem_notation,col_name): 
    spe_vob= codecs.open('SPE_ChEMBL_'+str(col_name)+'.txt')
    spe = SPE_Tokenizer(spe_vob)
    lst = spe.tokenize(chem_notation)
    lst = lst.split(' ')
    return lst

In [5]:
def removeBeginningInChI(chem_notation,substring):
    res = chem_notation.replace(substring,'')
    return res

In [6]:
def plotPredictReal(predictions,y_test,chem_notation,predictionsPlot_fileName):
        plt.figure(figsize=(10,10))
        plt.scatter(y_test, predictions, c='crimson')
        p1 = max(max( predictions), max(y_test))
        p2 = min(min( predictions), min(y_test))
        plt.plot([p1, p2], [p1, p2], 'b-')
        plt.title(chem_notation, fontsize=15)
        plt.xlabel('True Value', fontsize=15)
        plt.ylabel('Predictions', fontsize=15)
        plt.axis('equal')
        plt.show
        plt.savefig(predictionsPlot_fileName)

In [7]:
def saveModelLossPlot(result,chem_notation,modelLossPlot_fileName):
    # plot training history
    plt.figure(figsize=(10,10))
    plt.plot(result.history['loss'])
    plt.plot(result.history['val_loss'])
    plt.title('model loss - '+chem_notation)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper right')
    plt.savefig(modelLossPlot_fileName)

In [8]:
# kontrola, rozdeleni data tak je vzdy stejne i kdyz pouziji data pro jiny chem. lin. zapis
def splitData(list_token, ys, RANDOM_STATE):  
    X_train, X_test, y_train, y_test = train_test_split(list_token, ys, test_size=0.2, random_state=RANDOM_STATE)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE) # 0.25 x 0.8 = 0.2
    return X_train,y_train,X_val,y_val,X_test,y_test#,vocabulary_size

In [9]:
def replaceMultiWhitespaceSingle(x):
    _RE_COMBINE_WHITESPACE = re.compile(r"\s+")
    my_str = _RE_COMBINE_WHITESPACE.sub(" ", x).strip()
    y = [re.findall(r"\[(.+?)\]", my_str[1:])[i] for i in range(my_str.count('[')-1)]
    res = [re.findall(r"\d+\.\d+", y[i]) for i in range(len(y))]
    final_res = [list(map(float, i)) for i in res]
    return final_res

In [10]:
def multipleToSingleWhitespaceConvertStringToList(lst):
    _RE_COMBINE_WHITESPACE = re.compile(r"\s+")
    lst = _RE_COMBINE_WHITESPACE.sub(" ", lst).strip()
    lst = lst.replace('\'', "")
    lst = lst.strip('][').split(', ')       
    return lst

In [11]:
# vytvoreni modelu s tunnenim parametru
def create_modelPrams(params, X_train):
    opt = keras.optimizers.Adam(learning_rate=params['l_rate'])
    model = Sequential()
    # First layer specifies input_shape and returns sequences
    # model.add(LSTM(units=params['units1'], return_sequences=True, input_shape=(X_train.shape[1],X_train.shape[2])))
    model.add(Bidirectional(LSTM(units=params['units1']), input_shape=(X_train.shape[1],X_train.shape[2])))
    model.add(Dropout(rate=0.5))
    # Middle layers return sequences
    for i in range(params['layers']-2):
        model.add(Dense(units=params['units2']))
        model.add(Dropout(rate=0.5))

    # Last layer doesn't return anything
    model.add(Dense(units=params['units3']))
    model.add(Dropout(rate=0.5))

    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=opt ,metrics=['mse'])
    # model.summary()
    return model

In [12]:
# vytvoreni modelu s tunnenim parametru
def create_model(layers, units1, units2, units3, l_rate, cut_length,X_train):         
    opt = keras.optimizers.Adam(learning_rate=l_rate)
    model = Sequential()
    # First layer specifies input_shape and returns sequences
    # model.add(LSTM(units=params['units1'], return_sequences=True, input_shape=(X_train.shape[1],X_train.shape[2])))
    model.add(Bidirectional(LSTM(units=units1), input_shape=(X_train.shape[1],X_train.shape[2])))
    model.add(Dropout(rate=0.5))
    # Middle layers return sequences
    for i in range(layers-2):
        model.add(Dense(units=units2))
        model.add(Dropout(rate=0.5))

    # Last layer doesn't return anything
    model.add(Dense(units=units3))
    model.add(Dropout(rate=0.5))

    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=opt ,metrics=['mse'])
    
    return model

In [13]:
def builtModelWithBestParams(best_params,cut_length,cut_data,j,bestModel_fileName,modelHistory_fileName, bestParams_fileName,
                                 predictions_fileName,modelLossPlot_fileName,predictionsPlot_fileName,n_patience,encoding,chem_notation,
                                X_train,y_train,X_val,y_val,X_test,y_test):

    model = create_modelPrams(best_params,X_train)
    
    mc = ModelCheckpoint(bestModel_fileName, monitor='val_loss', mode='min')
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=n_patience)

    # train model
    result = model.fit(X_train, y_train, validation_data=(X_val, y_val),  epochs=400,verbose=0, callbacks=[es,mc])
    # store information
    # ----------------------------------------- BEST MODEL ---------------------------------------
    best_model = models.load_model(bestModel_fileName)
    # model prediction
    y_pred = best_model.predict(X_test)
    results_eval = best_model.evaluate(X_test, y_test)

    print('\nn_patience:  ', n_patience)
    print('max_length:  ',cut_length)
    print('results:',results_eval ,end='\n')
    # print(len(y_pred),len(y_val))
    plotPredictReal(y_pred, y_test,chem_notation,predictionsPlot_fileName)
    saveModelLossPlot(result,chem_notation,modelLossPlot_fileName)
    
    #store information
    # ----------------------------------------- HISTORY ---------------------------------------
    with open(modelHistory_fileName, 'w') as f:
        np.save(modelHistory_fileName,result.history)
    # ----------------------------------------- BEST PARAMS ---------------------------------------
    a_file = open(bestParams_fileName, "wb")
    pickle.dump(best_params, a_file)
    a_file.close()
    # ----------------------------------------- PREDICTIONS ---------------------------------------
    prediction = pd.DataFrame(y_pred, columns=['predictions']).to_csv(predictions_fileName)
    
    r2 = r2_score(y_test, y_pred)
    return [results_eval,r2]

### SPE2Vec

In [14]:
# objective funkce pro hyperopt s cross validaci
def objective(params, X_train, y_train, n_patience, idx, cut_length):
    es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=n_patience)
    
    model_nn = KerasRegressor(build_fn = create_model,**params, cut_length=cut_length, X_train = X_train,  epochs = 400, verbose = 0)
    
    kfold = KFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_val_score(model_nn, X_train, y_train,scoring='neg_mean_squared_error', cv=kfold,fit_params={'callbacks':[es]},error_score='raise')
    score = scores.mean()
    std = scores.std()
    loss = -score
    print('params:\t',params)
    print('score:\t',loss)
    print('std:\t',std)
    # Dictionary with information for evaluation
    return {
        'loss': loss,
        'params': params,
        'std' : std,
        'status': STATUS_OK }

In [15]:
def cutVecTokens(x,max_length):
    if(max_length >= len(x)):
        return x
    x = x[:max_length-1]
    return x

In [16]:
def tokenizeSPE(df,linearNotation_name,max_length):
    # odstranim zacatek retezce z inchi
    substring = 'InChI=1S/'
    df['inchi'] = df['inchi'].apply(lambda chem_notation: removeBeginningInChI(chem_notation,substring))
    col_name = linearNotation_name
    token_col_name = col_name+'_tokens'
    # definice tokenizatoru spe2 (prevadi na vysledny ciselny vektor)
    spe_vob= codecs.open('SPE_ChEMBL_'+str(col_name)+'.txt')
    spe = SPE_Tokenizer(spe_vob)
    spe2vec = SPE2Vec('../data/models/spe_model_'+col_name+'.bin', spe)
    # tokenizace SPE
    df[token_col_name] = df[col_name].apply(lambda chem_notation: tokenizeChemNotation(chem_notation,col_name))
    # stenym zpusobem jak v kodovani pred tim odstranim 
    df[token_col_name] = df[token_col_name].apply(lambda x: cutVecTokens(x,max_length))
    CHEM_NOTATION_CHARS = set(df[token_col_name].apply(list).sum())
    print(max_length)
    print('len(CHEM_NOTATION_CHARS)',len(CHEM_NOTATION_CHARS))

    if(len(CHEM_NOTATION_CHARS) * max_length > 100000):
        tempN = ((len(CHEM_NOTATION_CHARS) * max_length)-100000)/max_length
        n = math.floor(len(CHEM_NOTATION_CHARS)-tempN)
        n = len(CHEM_NOTATION_CHARS)-n

        temp = df[token_col_name].tolist()
        flat_list = [item for sublist in temp for item in sublist]
        count_flatList = Counter(flat_list)
        # list keyes, ktere pak vyhazujeme => odstranit ze sloupce s vektory tokeny
        keysSort = [pair[0] for pair in sorted(count_flatList.items(), key=lambda item: item[1],reverse=False)]
        
        print(n)
        for i in range(n):
            del count_flatList[keysSort[i]]
        CHEM_NOTATION_CHARS = count_flatList.keys()
        print('nova delka: ',len(CHEM_NOTATION_CHARS))
        
        df[col_name+'_tokens_clean'] = df[token_col_name].apply(lambda x: removeTokens(x,list(CHEM_NOTATION_CHARS)))
        df[col_name+'_clean'] = df[col_name+'_tokens_clean'] .apply(lambda x: "".join(x))
    else:
        df[col_name+'_clean'] = df[col_name]
    
    df[col_name+'_SPE2Vec'] = df[col_name+'_clean'].apply(lambda x: spe2vec.spe2vec(x))
    df[col_name+'_SPE2Vec'] = df[col_name+'_SPE2Vec'].apply(lambda x: editDataSameLength(x,max_length))

    return np.array(df[col_name+'_SPE2Vec'].tolist())

In [17]:
# tady e treba doplnit do vybrane delky, pridavame rovnou cely list (jako token)
def editDataSameLength(x,cut_length):
    res=[]
    lengthTokenSpe=100 # jednotlive tokeny jsou zakodovany do delky 100 v SPE2 (3. dimenze)
    zeroVec= np.array([[0] * lengthTokenSpe]) # vytvorim vnulovy vektor

    res=np.array([np.array(i) for i in x[:cut_length] if not isinstance(i, float) ])
    
    if(not res.any()):
        res  = np.array( zeroVec )
    numberTokens = len(res)
        
    if(cut_length > numberTokens):
        for i in range(numberTokens,cut_length):
            res = np.append(res,zeroVec, axis = 0)    
    return res

In [18]:
def removeTokens(x,CHEM_NOTATION_CHARS):
    cleanVec=[]
    for i in range(len(CHEM_NOTATION_CHARS)):
        for value in x:
            if(value == CHEM_NOTATION_CHARS[i]):
                 cleanVec.append(value)
    return cleanVec

## Trenovani neuronove site

In [ ]:
# poradi podle chem. lin. zapisu
linearNotation_names = ['canonical_smiles','sln','deep_smiles','selfies','inchi']
    
# order: canonical_smiles,sln,deep_smiles,selfies,inchi    
lengths_lst_tokenIdxs = [13,8,7,31,43]

RANDOM_STATE=42
n_patience=20
m_e=30
encoding = 'SPE2Vec'

ys = np.array(dfClean['value'].tolist())#[:limit]

df_score = {'chem_notation': ['empty'],'score_cross_val':['empty'],'std_cross_val':['empty'],'loss_mse':['empty'],'metric_mse':['empty'],'r2':['empty']}
df_score = pd.DataFrame(df_score)

fileNameScore = '../data/final_results/'+encoding+'/scoreCrossVal.csv'
# kontrola, jestli soubor neexistuje, kdyz poustim znova
# if os.path.exists(fileNameScore):
#     os.remove(fileNameScore)
# df_score.to_csv(fileNameScore,sep=',',header=True,index=False)

# pro hyperopt a jeho volba poctu neuronu
or_unitsLst = [16, 32, 64, 128]

for i in range(len(linearNotation_names)):
    chem_notation = linearNotation_names[i]
    # df_score['chem_notation'][0] = chem_notation
    if(chem_notation=='canonical_smiles'):
        continue
    print('-----------------------------------------------------------------------------\n'+chem_notation)
    df_score['chem_notation'][0] = chem_notation
    
    difSmallNum=5000
    
    # vyzkouset neuronovou sit, jestli bude schopna predikovat cely rozsah cilove promenne
    cut_length=lengths_lst_tokenIdxs[i]
    
    # ZKONTROLOVAT A PRIPADNE ZMENIT
    # vybrani poctu neuronu pro vstupni vrstvu
    for u in range(len(or_unitsLst)):
        if(or_unitsLst[u]>cut_length):
            difGreatNum = abs(or_unitsLst[u] - cut_length)
            if(u!=0):
                difSmallNum = abs(or_unitsLst[u-1] - cut_length)
            if(difGreatNum <= difSmallNum):    
                nUnitsInputLayer = [or_unitsLst[u]]
                nUnitsOtherLayers = or_unitsLst[:(u+1)]
                break
            else:
                nUnitsInputLayer = [or_unitsLst[u-1]]
                nUnitsOtherLayers = or_unitsLst[:(u)]
                break
        elif(or_unitsLst[u]==cut_length or u==(len(or_unitsLst)-1)):
            nUnitsInputLayer = [or_unitsLst[u]]
            nUnitsOtherLayers = or_unitsLst[:(u+1)]
            break
    # print(cut_length)
    # print(nUnitsInputLayer)
    # print(nUnitsOtherLayers)

    space = {'units1'     : hp.choice('units1', nUnitsInputLayer),
         'units2'             : hp.choice('units2', nUnitsOtherLayers),
         'units3'             : hp.choice('units3', nUnitsOtherLayers),
         'layers'             : scope.int(hp.quniform('layers',2,3,1)),
         'l_rate'             : hp.loguniform('l_rate', np.log(0.01), np.log(0.2))
        }
    
    data = tokenizeSPE(dfClean.copy(),chem_notation,cut_length)
    print(data.shape)
    
    X_train,y_train,X_val,y_val,X_test,y_test = splitData(data, ys, RANDOM_STATE)

    # definice nazvu pro ukladani
    startName = '../data/final_results/'+encoding+'/'
    bestModel_fileName = startName+'bestModels/'+chem_notation+'.h5'
    modelHistory_fileName = startName+'history/'+chem_notation+'.npy'
    modelLossPlot_fileName = startName+'lossPlot/'+chem_notation+'.png'
    bestParams_fileName = startName+'bestParams/'+chem_notation+'.pkl'
    predictions_fileName = startName+'predictions/'+chem_notation+'.csv'
    predictionsPlot_fileName = startName+'predictionsPlot/'+chem_notation+'.png'

    # zatim mam jen chemicke linearni zapisy
    fmin_objective = partial(objective, n_patience = n_patience, idx=i,cut_length = cut_length,X_train=X_train,y_train=y_train)
    bayes_trials = Trials()
    best = fmin(fn = fmin_objective, space = space, algo = tpe.suggest, max_evals = m_e, trials = bayes_trials)#, rstate = np.random.RandomState(50))
    
    best_loss = bayes_trials.results[np.argmin([r['loss'] for r in bayes_trials.results])]['loss']
    best_std = bayes_trials.results[np.argmin([r['loss'] for r in bayes_trials.results])]['std']
    best_params = bayes_trials.results[np.argmin([r['loss'] for r in bayes_trials.results])]['params']
    print('best params:\n',best_params,'\nbest loss:\n',best_loss,end = '\n')
    
    df_score['score_cross_val'][0] = best_loss
    df_score['std_cross_val'][0] = best_std

    results_eval,r2 = builtModelWithBestParams(best_params,cut_length,data,i,bestModel_fileName,modelHistory_fileName, bestParams_fileName,
                             predictions_fileName,modelLossPlot_fileName,predictionsPlot_fileName,n_patience,encoding,chem_notation,
                             X_train,y_train,X_val,y_val,X_test,y_test)
    
    df_score['loss_mse'][0] = results_eval[0]
    df_score['metric_mse'][0] = results_eval[1]
    df_score['r2'][0] = r2
    df_score.to_csv(fileNameScore, mode='a', index=False, header=False)

-----------------------------------------------------------------------------
sln
8
len(CHEM_NOTATION_CHARS) 5760
(7044, 8, 100)
  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

2022-03-12 08:05:03.233873: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2022-03-12 08:05:03.233912: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-03-12 08:05:03.234263: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 00063: early stopping                           
Epoch 00076: early stopping                           
Epoch 00072: early stopping                           
Epoch 00051: early stopping                           
Epoch 00098: early stopping                           
Epoch 00056: early stopping                           
Epoch 00069: early stopping                           
Epoch 00096: early stopping                           
Epoch 00055: early stopping                           
Epoch 00047: early stopping                           
params:	                                              
{'l_rate': 0.023337399477386594, 'layers': 3, 'units1': 16, 'units2': 16, 'units3': 16}
score:	                                               
0.8298869305324332                                    
std:	                                                 
0.05518802442469624                                   
Epoch 00045: early stopping                                                        
Epo

In [134]:
# for i in range(20):
#     print(data[i])

In [ ]:
#  # for np in n_patiences:
#         model = create_modelPrams(params, X_train)
        
#         startName = '../data/models/'+encoding
#         bestModelName = startName +'/bestModel_otherArchitecture_'+chem_notation+'_nt_'+str(nt)+'_np_'+str(n_patience)+'.h5'
#         predictionsPlot_fileName = startName +'/predictionsPlot_'+chem_notation+'_nt_'+str(nt)+'_np_'+str(n_patience)+'.png'

#         mc = ModelCheckpoint(bestModelName, monitor='val_loss', mode='min')
#         es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=n_patience)
#         # validation_data=(X_val, y_val)
#         X_train, X_val_fit, y_train, y_val_fit = train_test_split(X_train,y_train,test_size=0.2,random_state=RANDOM_STATE) # before model building
#         result = model.fit(X_train, y_train, validation_data=(X_val_fit, y_val_fit),  epochs=400,verbose=0, callbacks=[es,mc])
#         best_model = models.load_model(bestModelName)
#         # # model prediction
#         y_pred = best_model.predict(X_val)
#         results_eval = best_model.evaluate(X_val, y_val)
#         print('\nn_patience:  ', n_patience)
#         print('cut_length:  ',cut_length)
#         print('results:',results_eval ,end='\n')
#         # print(len(y_pred),len(y_val))
#         plotPredictReal(y_pred,y_val,chem_notation,predictionsPlot_fileName)

### SLN

### Deep smiles

### SELFIES

### InChI

### SELFIES